In [ ]:
# Load the dataset from JSONL.GZ file
import pandas as pd
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from transformers import pipeline

file_path = r"C:\Users\megap\Desktop\Projeto\Books.jsonl.gz"

# Load up to 100,000 lines from the gzipped JSONL file
data = []
with gzip.open(file_path, "rt", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 100_000:
            break
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError:
            continue

df = pd.DataFrame(data)
print(df.head())
print(df.columns)
print(df.info())

In [ ]:
tqdm.pandas()
distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    return_all_scores=True
)

def get_predicted_label(text):
    preds = distilled_student_sentiment_classifier(text[:512])[0]
    return max(preds, key=lambda x: x["score"])["label"]

df["predicted_sentiment"] = df["text"].progress_apply(get_predicted_label)

In [ ]:
def rating_to_sentiment(score):
    if score >= 4:
        return "positive"
    elif score == 3:
        return "neutral"
    else:
        return "negative"

df["sentiment"] = df["rating"].apply(rating_to_sentiment)

from sklearn.metrics import classification_report
print(classification_report(df["sentiment"], df["predicted_sentiment"]))



In [ ]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["text"].apply(clean_text)


df["sentiment"].value_counts()
min_class_size = df["sentiment"].value_counts().min()
balanced_df = df.groupby("sentiment").apply(lambda x: x.sample(min_class_size, random_state=42)).reset_index(drop=True)
print(balanced_df["sentiment"].value_counts())

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=) 

def cluster_texts_zero_shot(texts, max_texts_per_prompt=20):
    categoria_por_texto = {}
    for i in range(0, len(texts), max_texts_per_prompt):
        batch = texts[i:i+max_texts_per_prompt]

        prompt = """
Aqui está um exemplo de como agrupar textos em categorias:

Categoria 1: Romance Histórico  
Textos: [1, 3]
Categoria 2: Autoajuda  
Textos: [2, 4]
Categoria 3: Ficção Científica  
Textos: [5]
Categoria 4: Livros Infantis  
Textos: [6]
Categoria 5: Ensaios Acadêmicos  
Textos: [7]

Agora agrupa estes textos em **exatamente 5 categorias temáticas distintas**.  
Dá a cada categoria um nome curto, claro e representativo. De seguida, indica os textos que pertencem a cada categoria.

Textos:
"""
        for idx, text in enumerate(batch):
            prompt += f"{idx+1}. {text[:200]}...\n"

        prompt += """
Responde no seguinte formato:
Categoria 1: [nome]
Textos: [números]
Categoria 2: [nome]
Textos: [números]
Categoria 3: [nome]
Textos: [números]
Categoria 4: [nome]
Textos: [números]
Categoria 5: [nome]
Textos: [números]
"""

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4
        )

        answer = response.choices[0].message.content
        categoria_atual = None
        for linha in answer.strip().split("\n"):
            if linha.startswith("Categoria"):
                categoria_atual = linha.split(":")[1].strip()
            elif linha.startswith("Textos") and categoria_atual:
                try:
                    indices = eval(linha.split(":")[1].strip())
                    for idx in indices:
                        global_idx = i + idx - 1
                        categoria_por_texto[global_idx] = categoria_atual
                except:
                    continue

    return categoria_por_texto

categorias_map = cluster_texts_zero_shot(balanced_df["clean_text"].tolist())
balanced_df["categoria_original"] = balanced_df.index.map(categorias_map)

In [ ]:
import unicodedata

def normalizar_texto(texto):
    texto = str(texto).lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = re.sub(r"[^a-zA-Z0-9\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

balanced_df = balanced_df.dropna(subset=["categoria_original"]).reset_index(drop=True)
balanced_df["categoria_limpa"] = balanced_df["categoria_original"].apply(normalizar_texto)

# Reclustering categories with TF-IDF and KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

cat_counts = balanced_df["categoria_limpa"].value_counts()
categorias_frequentes = cat_counts[cat_counts >= 5].index.tolist()
balanced_df = balanced_df[balanced_df["categoria_limpa"].isin(categorias_frequentes)]

categories = balanced_df["categoria_limpa"].unique().tolist()
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(categories)

num_clusters = min(10, len(categories))
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(X)

cat_freqs = balanced_df["categoria_limpa"].value_counts().to_dict()
category_map = {}
for cluster_num in range(num_clusters):
    cluster_cats = [cat for cat, label in zip(categories, labels) if label == cluster_num]
    main_cat = max(cluster_cats, key=lambda c: cat_freqs.get(c, 0))
    for cat in cluster_cats:
        category_map[cat] = main_cat

balanced_df["categoria_normalizada"] = balanced_df["categoria_limpa"].map(category_map).fillna(balanced_df["categoria_limpa"])



In [ ]:
import textwrap

def resumir_textos_openai(textos, categoria, max_texts=100):
    joined_text = " ".join(textos[:max_texts])[:4000]
    prompt = f"""
Faz um resumo curto e representativo dos seguintes comentários de clientes sobre livros da categoria \"{categoria}\".

Comentários:
{joined_text}

Resumo:"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=200
    )
    return response.choices[0].message.content.strip()

resumos_por_categoria = {}
for cat in balanced_df["categoria_normalizada"].unique():
    textos_cat = balanced_df[balanced_df["categoria_normalizada"] == cat]["clean_text"].tolist()
    resumo = resumir_textos_openai(textos_cat, categoria=cat)
    resumos_por_categoria[cat] = resumo
    print(f"\n🔹 Summary for '{cat}':\n{textwrap.fill(resumo, width=100)}")

In [ ]:
def recomendar_livros_por_categoria(df, categoria, top_n=5):
    subset = df[df["categoria_normalizada"] == categoria]
    if subset.empty:
        print("Categoria não encontrada.")
        return []
    agrupado = subset.groupby("title").agg({"rating": "mean", "text": "first", "asin": "first"}).reset_index()
    top_livros = agrupado.sort_values(by="rating", ascending=False).head(top_n)
    return top_livros

In [ ]:
def mostrar_top_livros_por_categoria(df, categoria, top_n=5, min_rating=4):
    subset = df[(df["categoria_normalizada"] == categoria) & (df["rating"] >= min_rating)]
    subset = subset.sort_values(by="rating", ascending=False).head(top_n)

    if subset.empty:
        print(f"Nenhum livro encontrado para a categoria '{categoria}' com rating >= {min_rating}.")
        return

    print(f"\n📚 Top {top_n} livros da categoria: {categoria}\n")
    for _, row in subset.iterrows():
        print(f"📘 {row['title']} ({row['rating']:.1f} estrelas)")
        print("📝 Exemplo de crítica:")
        print(textwrap.fill(row["text"], width=100))
        print("\n" + "-"*80 + "\n")

categoria_escolhida = "romance"
mostrar_top_livros_por_categoria(balanced_df, categoria_escolhida, top_n=5)
